In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
from math import log
import csv
import string 

In [48]:
#Portfolio Extraction

data = pd.DataFrame(columns=['Ticker','Market Cap','Benchmark Proportion'])
ticker_name= []
#Reads ticker_name.csv: Column of Ticker Name, Column of Expexted Investor Returns in %
with open("ticker_name.csv", errors="ignore") as f:
    reader = csv.reader(f, delimiter=",")
    for i in reader:
        if i != "":
            ticker_name.append(i[0]+".NS")

for i in range(len(ticker_name)):
    if ticker_name[i].strip() != ".NS":
        data.loc[i, "Ticker"]= ticker_name[i] 
for i in range(len(data['Ticker'].values)):
    if(data['Ticker'].values[i].lstrip() != ".NS"):
        ticker_data = yf.download(data['Ticker'].values[i], start=(datetime.today()+ relativedelta(days = -60)).strftime('%Y-%m-%d'), end=datetime.today().strftime('%Y-%m-%d'), interval='1d')
        ticker_info = yf.Ticker(data['Ticker'].values[i])
        try:
            data.loc[i, "Market Cap"] = ticker_info.info['marketCap']
        except Exception: 
            pass
        tick_data = ticker_data["Close"]
        for j in range(len(tick_data)):
            close = tick_data.values[j][0]
            col_name = "Price " + str(j+1)
            try:
                data.insert(j+3, col_name, close)
            except: 
                data.at[i , col_name] = close
for i in range(len(data)):
    data.loc[i, 'Benchmark Proportion'] = data.loc[i, 'Market Cap']/data['Market Cap'].sum()
        
data.to_excel('data.xlsx')
#After execution of this block, 
# 1. Open data.xlsx and delete the index row.
# 2. Use text to columns to seperate the list of close prices to different cells.
# 3. Remove all [] brackets using CTRL+H
# 4. Raname Returns column as Price 1, and then name all other prices as Price 2 to Price 43(currently)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [61]:
#Variance-Covariance Matric Calculation


price_data = pd.read_excel("data.xlsx")
col_size = price_data.shape[1]
row_size = price_data.shape[0]
returns = []
print(col_size)
#Calculates log returns into returns
for i in range(0, row_size):
    entry = []
    for j in range(3, col_size-2):
        num = price_data.loc[i, 'Price ' + str(j-2)]
        denom = price_data.loc[i, 'Price ' + str(j-1)]
        entry.append((log(num/denom)))
    returns.append(entry)

varcov = np.cov(returns)
inverse = np.linalg.inv(varcov)

45


In [63]:
#Normalizing Factor Calculation
prop = data['Benchmark Proportion']
bench_prop = np.matrix(prop)   #Benchmark Proportions, Market share of stock i in the portfolio
a_ret = 1     #Anticipated returns 
rfr = 0.127   #Risk Free Rate
benchprop_t = bench_prop.transpose()

norm_fact = (a_ret-rfr)/(np.dot(bench_prop, np.dot(varcov, benchprop_t)))
print(norm_fact)

[[6376.216334473713]]


In [67]:
#Expected Benchmark Return with no investor opinion
exp_benchret_no = np.dot(varcov, benchprop_t)*norm_fact+rfr
print(exp_benchret_no)

[[0.8860340185422027]
 [0.6474045119118751]
 [0.9561405925589757]
 [1.1754955866647097]
 [0.9416131999316781]
 [1.167678510603784]
 [0.6120261696873719]
 [-0.0727636299828264]
 [0.6337139302232446]
 [0.2299622213695876]
 [0.6780572757524203]
 [0.5188475615174468]
 [1.1937025925518074]
 [0.8756833101033864]
 [0.3005140554519468]
 [0.4614548807800306]
 [0.11689480974429449]
 [0.9724479043302837]
 [0.9166494788294807]
 [0.651358965442323]]


In [69]:
#optimized Portfolio with no investor opinion
opt_port_no = np.dot(inverse,exp_benchret_no-rfr)/(np.dot(inverse,exp_benchret_no-rfr)).sum()
print(opt_port_no)

[[0.004715821847801723]
 [0.16470493761054053]
 [0.012267774930402963]
 [0.003369957693634626]
 [0.041883160594649485]
 [0.6450769631473057]
 [0.007509217451929425]
 [0.0016632470295588745]
 [0.0005006076256095537]
 [0.001954838003138791]
 [0.0042415869614073265]
 [0.005433194977035705]
 [0.004270862725347029]
 [0.03063069778556192]
 [0.008527065280101899]
 [0.017581424128034078]
 [0.011591905130816557]
 [0.007290520939984482]
 [0.005255901154086398]
 [0.02153031498305275]]


In [71]:
# Expected Benchmark Returns with Investor Opinion
blm_matrix = np.matrix(varcov/np.var(returns))
inv_opinion = []
with open("ticker_name.csv", errors="ignore") as f:
    reader = csv.reader(f, delimiter=",")
    for i in reader:
        if i != "":
            inv_opinion.append(float(i[1]))
#inv_op_matrix = np.matrix(inv_opinion, data = 'float')
opinion_returns = exp_benchret_no + np.dot(inv_opinion, blm_matrix).transpose()
print(opinion_returns)

[[1.6569269839128835]
 [0.9703854970263925]
 [1.575471059080927]
 [2.1621699572452933]
 [2.136661539863842]
 [1.49077574975975]
 [1.3219215775933644]
 [0.2684269736757746]
 [1.4864635639353452]
 [0.6740779600446265]
 [1.4960049332416094]
 [1.0455427802382293]
 [1.935354826502699]
 [1.5337930489808045]
 [0.7575153391229174]
 [0.7372247173310844]
 [0.3732158755068965]
 [1.8279611980705783]
 [1.9826066016037855]
 [1.2459141120263928]]


In [73]:
#Optimized Portfolio adjusted for Investor Opinion
opt_port_weights = np.dot(inverse,opinion_returns-rfr)/(np.dot(inverse,opinion_returns-rfr)).sum()
print(opt_port_weights)

[[0.004551151110948014]
 [0.16433462709748595]
 [0.0072856342994345805]
 [0.002001363694694315]
 [0.10189575035772572]
 [0.39185333903240566]
 [0.00796060418305103]
 [0.020243279331815005]
 [0.023053807147520148]
 [0.023917451188751198]
 [0.025275514630498264]
 [0.02598319147323433]
 [0.01478989903892917]
 [0.03394558195399093]
 [0.010315587976887912]
 [0.020944327126294186]
 [0.01738724808866936]
 [0.0585952334932889]
 [0.024127399189326625]
 [0.02153900958504838]]
